In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import rrule
import ahmedsabri
from ahmedsabri import *
%matplotlib inline

In [2]:
start_date = datetime(2022, 1, 1) # year,month,day
end_date = datetime(2023, 2, 25) # year, month, day

In [3]:
def prepare(data):
    df=data.copy()
    df=df.transpose()
    df.columns=df.iloc[0:3].fillna('').astype(str).apply(' '.join).str.strip()
    df=df.iloc[3:]
    #df.columns=df.columns.str.replace(" ","_")
    #df.columns=df.columns.str.lower()
    #df.columns=df.columns.str.replace("-","_")
    df.reset_index(drop=True, inplace=True)
    #df.dropna(how="any", thresh=df.shape[0]*0.99,inplace=True)
    #df.dropna(axis="columns", how="any", thresh=df.shape[0]*0.99, inplace=True)
    #df=df.replace('No Data',np.nan)
    #df=df.replace('No Lab Data',np.nan)
    #df=df.replace('NaT',np.nan)
    #df=df.replace('-',np.nan)
    df=df.replace('Nil',0)
    return df

In [4]:
def to_float(df):    
    for column in df.columns:
        try:
            df[column]=df[column].astype(float)
        except:
            continue
    return df

In [5]:
def to_numbers(df):
    '''
    converting dataframe to numeric values, all other non-numbers will converted to Nan
    '''
    for column in df.columns:
        df[column]=pd.to_numeric(df[column],errors='coerce')
    return df

In [6]:
def plotting(df):
    for i in df.columns:
        try:
            fig, ax = plt.subplots(figsize=(7,3))
            ax.plot(df[i])
            ax.set_ylabel(i)
            ax.set_xlabel('days')
            plt.xticks(rotation=45)
            plt.show()
        except:
            pass   

In [7]:
def date_plot(df,date):
    for column in df.columns:
        try:
            fig,ax=plt.subplots(figsize=(10,5))
            sns.scatterplot(x=date['date'],y=df[column],ax=ax)
            plt.xticks(rotation=45)
            plt.show()
        except:
            pass

In [8]:
def outlier_columns(df,a=4):
    z_scores = stats.zscore(df[df.describe(exclude=[datetime]).columns])
    z_scores.fillna(0,inplace=True)
    abs_z_scores = np.abs(z_scores)
    (abs_z_scores>a).any(axis=0)
    outliers_columns=abs_z_scores.columns[(abs_z_scores>a).any(axis=0)]
    return df[outliers_columns]

In [9]:
def lab_assurance(df):
    for i in range (1,411):
        if type(df.iloc[i,0])!=str:
            df.iloc[i,0]=df.iloc[i-1,0]
    #df.dropna(axis=0,how='any',thresh=6,inplace=True)
    #df.reset_index(inplace=True,drop=True)

In [10]:
# making dictionary which keys is month_year and values are the dataframes
df_macro_dict={}
df_lab_dict={}
for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
    os.chdir(os.path.join(r'\\192.168.21.16\Area 02A sharing\Spread Sheets',str(dt.year)))
    datetime_object = datetime.strptime(str(dt.month), "%m")
    month = datetime_object.strftime("%b")
    year=str(dt.year)[-2:]
    sheet_name='Unit-12 & 16 SS' +' '+ month.capitalize() +' '+ year+'.xls'
    #df='df'+'_'+str(dt.month)+'_'+str(dt.year)[-2:]
    df_macro_dict[month+'_'+year]=pd.read_excel(sheet_name ,header = None,sheet_name='Macro U-12',usecols='a:ah',nrows=141)
    df_lab_dict[month+'_'+year]=pd.read_excel(sheet_name,header = None,sheet_name="Lab Summary U-12",usecols="a:ah",nrows=238)

## prepare macro and concacenate it

In [11]:
for key in df_macro_dict.keys():
    df_macro_dict[key]=prepare(df_macro_dict[key])
    df_macro_dict[key]['Tag Macro Data']=pd.to_datetime(df_macro_dict[key]['Tag Macro Data'],errors='coerce')
    if df_macro_dict[key].columns.duplicated().sum()>0:
        df_macro_dict[key] = df_macro_dict[key].loc[:,~df_macro_dict[key].columns.duplicated()].copy()
    df_macro_dict[key].set_index('Tag Macro Data',inplace=True)

In [12]:
df_macro=pd.concat(df_macro_dict.values(),axis=0)

## preparing Lab and concacenate it

In [13]:
for key in df_lab_dict.keys():
    #initiating new column that contains only SC and all other are nans
    df_lab_dict[key]['X'] = df_lab_dict[key][0].where(df_lab_dict[key][0].str.contains('sc',case=False,regex=True), np.nan)
    #fill forward the nans
    df_lab_dict[key]['X'] = df_lab_dict[key]['X'].fillna(method='ffill')
    #adding both column to the original one
    df_lab_dict[key][0]= df_lab_dict[key]['X']+'_'+df_lab_dict[key][0]
    #dropping the new column
    df_lab_dict[key].drop('X', axis=1, inplace=True)
    df_lab_dict[key]=prepare(df_lab_dict[key])
    df_lab_dict[key]['Description_Description']=pd.to_datetime(df_lab_dict[key]['Description_Description'],errors='coerce')
    if df_lab_dict[key].columns.duplicated().sum()>0:
        df_lab_dict[key] = df_lab_dict[key].loc[:,~df_lab_dict[key].columns.duplicated()].copy()
    df_lab_dict[key].set_index('Description_Description',inplace=True) 

In [14]:
#https://www.geeksforgeeks.org/merge-two-dataframes-with-same-column-names/
df_lab=pd.concat(df_lab_dict.values(),axis=0)

# MERGING and saving to CSV

In [15]:
df_macro.shape

(1178, 131)

In [16]:
df_lab.shape

(1178, 217)

In [17]:
df_macro

,Daily Monitoring Data Units,Description,12-FI-001 Sour Gas To 12 T01 NM3/H,12-TI-001 Temp.Of Incoming Sour Gas o C,12-PI-066 Press. Of Incoming Sour Gas bar g,12-FIC-002 Lean MDEA To T01 (sour gas) M3/H,12-TI-004 Temp. Of Lean MDEA To T01 o C,TI 002 Temp. Of Rich MDEA From T01 o C,LIC 002 Level Of Rich MDEA In T01 To U-14 %,LIC 003 Level Of Rich MDEA In T01 To U-14 %,...,08FI094 LPG from P-101A/B m3/hr,08TI390 Washed LPG temp. from V-102 to B.L. °C,11-FIC-068 C3/C4 PRODUCT TO B/L m3/hr,11-TI-165 LPG product to U -12 °C,12-LDV-032,12-PV-032,12-PY-061A,12-PY-061B,12-HIC-001,39-PI-004
Tag Macro Data,,,,,,,,,,,,,,,,,,,,,
2020-01-01,Need Some Help,Ok,25765.887451,40.0,6.393296,142.943695,51.233212,61.587301,25.293002,25.646806,...,6.198429,30.23199,7.129336,38.852259,73.87613,2.142992,96.25,0,0,3.571823
2020-01-02,NaN,Ok,26679.13501,39.0232,6.404179,139.224595,50.875052,62.026862,25.298229,28.659153,...,6.939788,29.43427,7.276688,38.754579,73.896449,1.406059,96.25,0,0,3.574258
2020-01-03,NaN,Ok,25792.515869,37.826618,6.389582,139.093316,49.18193,59.902319,25.289977,24.396382,...,7.266058,28.522589,7.027377,38.608059,73.11905,-0.186442,96.25,0,0,3.572448
2020-01-04,NaN,Ok,15551.719671,38.168499,6.304897,139.426003,49.39357,54.554334,26.879973,24.202916,...,3.803646,24.582825,5.319275,35.286936,73.240131,1.992438,96.25,0,0,3.558319
2020-01-05,NaN,Ok,6767.100375,39.12088,6.240256,145.172824,49.768011,51.672772,28.085774,24.89622,...,0.960472,14.847375,2.527602,27.765568,73.839538,0.260771,96.25,0,0,3.553597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-27,NaN,Ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-28,NaN,Ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,Ok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_macro = df_macro[~df_macro.index.duplicated(keep="first")]

In [19]:
df_lab = df_lab[~df_lab.index.duplicated(keep="first")]

In [20]:
df=pd.concat([df_macro, df_lab], axis=1)

In [21]:
os.chdir(r'\\192.168.21.16\Area 02A sharing\Spread Sheets\spread sheets database by python')

In [22]:
df.to_csv(f'U_12 spreadsheet{str(start_date).split()[0]}_to_{str(end_date).split()[0]}.csv')

## Converting all data to numbers

In [23]:
# to convert object only ( not dates or numbers)
df[df.dtypes[df.dtypes=='object'].index].columns

Index(['Daily Monitoring Data Units', 'Description',
       '12-FI-001 Sour Gas To 12 T01 NM3/H',
       '12-TI-001 Temp.Of Incoming  Sour Gas  o C',
       '12-PI-066 Press. Of Incoming Sour Gas  bar g',
       '12-FIC-002 Lean MDEA To T01 (sour gas) M3/H',
       '12-TI-004 Temp. Of Lean MDEA To T01 o C',
       'TI 002 Temp. Of Rich MDEA From T01 o C',
       'LIC 002 Level Of Rich MDEA In T01 To U-14 %',
       'LIC 003 Level Of Rich MDEA In T01 To U-14 %',
       ...
       '39-SC-001 Fuel gas_ BTU's Gross',
       '39-SC-001 Fuel gas_Rel. Dens. Air=1', '39-SC-001 Fuel gas_Ave. M.WT.',
       '12-SC-15 LPG amine absorber_12-SC-15 LPG amine absorber',
       '12-SC-15 LPG amine absorber_H2S',
       '12-SC-13 Caustic in LPG treater V-52_12-SC-13 Caustic in LPG treater V-52',
       '12-SC-13 Caustic in LPG treater V-52_Total Alkalinity',
       '12-SC-13 Caustic in LPG treater V-52_NaOH',
       '12-SC-13 Caustic in LPG treater V-52_Spent Caustic',
       '12-SC-13 Caustic in LPG t

In [24]:
df.shape

(1126, 348)

In [25]:
df= df.loc[:,~df.columns.duplicated()]

In [26]:
df.shape

(1126, 347)

In [27]:
for column in df[df.dtypes[df.dtypes=='object'].index].columns:
    df[column]=pd.to_numeric(df[column],errors='coerce')

In [28]:
df.to_csv('orange_data.csv')

In [29]:
os.getcwd()

'\\\\192.168.21.16\\Area 02A sharing\\Spread Sheets\\spread sheets database by python'